## Load the data

In [6]:
import os
import json


def load_data_by_language(
    embeddings_directory: str, language: str = "english"
) -> tuple[list[dict], list[dict]]:
    chunks, questions = [], []
    chunk_id_counter = 0
    file_paths = get_file_paths(embeddings_directory, language, sort=True)

    for file_path in file_paths:
        with open(file_path, "r") as f:
            data = json.load(f)

        old_to_new_id_map = {}
        file_chunks = data["chunks"]
        for chunk in file_chunks:
            old_id = chunk["id"]
            new_id = chunk_id_counter
            old_to_new_id_map[old_id] = new_id
            chunk["id"] = new_id
            chunks.append(chunk)
            chunk_id_counter += 1

        file_questions = data["question_answer_pairs"]
        for question in file_questions:
            old_chunk_id = question["chunk_id"]
            if old_chunk_id in old_to_new_id_map:
                question["chunk_id"] = old_to_new_id_map[old_chunk_id]
                questions.append(question)

    print(
        f"{language.capitalize()} - Chunks: {len(chunks)}, Questions: {len(questions)}"
    )
    return chunks, questions


def get_file_paths(embeddings_directory: str, language: str, sort: bool = False):
    file_paths = []

    for file in os.listdir(embeddings_directory):
        if not file.endswith(".json"):
            continue

        if language == "english":
            languages = ["arabic"]
            if not any(language in file.lower() for language in languages):
                file_paths.append(os.path.join(embeddings_directory, file))
        else:
            if language.lower() in file.lower():
                file_paths.append(os.path.join(embeddings_directory, file))

    if sort:
        file_paths.sort()

    return file_paths


embeddings_directory = "../data/embeddings/"
chunks_arabic, questions_arabic = load_data_by_language(embeddings_directory, "arabic")
chunks_english, questions_english = load_data_by_language(
    embeddings_directory, "english"
)

Arabic - Chunks: 120, Questions: 1209
English - Chunks: 120, Questions: 1209


## Benchmarks

In [ ]:
import numpy as np
import string

from typing import Any

from rich.box import HEAVY
from rich.text import Text
from rich.table import Table
from rich.console import Console

from ranx import Run, Qrels, compare
from ranx.data_structures.report import Report
from sklearn.metrics.pairwise import cosine_similarity


def create_qrels_from_data(questions: list[dict]) -> Qrels:
    qrels_dict = {}
    for i, question in enumerate(questions):
        query_id = f"q_{i}"
        correct_chunk_id = question["chunk_id"]
        qrels_dict[query_id] = {str(correct_chunk_id): 1}

    return Qrels(qrels_dict)


def create_run_from_data(
    chunks: list[dict], questions: list[dict], model_name: str
) -> Run:
    run_dict = {}

    chunk_embeddings = np.array([chunk["embeddings"][model_name] for chunk in chunks])
    question_embeddings = np.array(
        [question["embeddings"][model_name] for question in questions]
    )
    similarity_scores = cosine_similarity(question_embeddings, chunk_embeddings)

    chunk_ids = [str(chunk["id"]) for chunk in chunks]
    for i, scores_for_one_question in enumerate(similarity_scores):
        chunk_scores = {
            chunk_id: score
            for chunk_id, score in zip(chunk_ids, scores_for_one_question)
        }

        query_id = f"q_{i}"
        run_dict[query_id] = chunk_scores

    return Run(run_dict, name=model_name)


def print_benchmark_report(report: Report) -> None:
    model_to_letter = {}
    for letter, name in zip(string.ascii_lowercase, report.model_names):
        model_to_letter[name] = letter

    report.rounding_digits = 4
    cell_data = {}
    max_widths = {metric: len(metric) for metric in report.metrics}

    for current_model_name in report.model_names:
        cell_data[current_model_name] = {}
        for metric in report.metrics:
            superscripts = ""
            current_model_score = report.results[current_model_name][metric]

            for other_model_name in report.model_names:
                if current_model_name == other_model_name:
                    continue

                comparison_key = {current_model_name, other_model_name}
                p_value = report.comparisons[comparison_key][metric]["p_value"]
                other_model_score = report.results[other_model_name][metric]

                if current_model_score > other_model_score and p_value < report.max_p:
                    superscripts += model_to_letter[other_model_name]

            score_str = f"{current_model_score:.{report.rounding_digits}f}"
            sorted_superscripts = "".join(sorted(superscripts))
            cell_data[current_model_name][metric] = (score_str, sorted_superscripts)

            total_len = len(score_str)
            if sorted_superscripts:
                space_len = 1
                total_len += space_len + len(sorted_superscripts)

            if total_len > max_widths[metric]:
                max_widths[metric] = total_len

    max_scores = {metric: -float("inf") for metric in report.metrics}
    min_scores = {metric: float("inf") for metric in report.metrics}
    for model_name in report.model_names:
        for metric in report.metrics:
            score = report.results[model_name][metric]
            if score > max_scores[metric]:
                max_scores[metric] = score
            if score < min_scores[metric]:
                min_scores[metric] = score

    console = Console()
    table = Table(show_header=True, header_style="bold bright_magenta", box=HEAVY)
    table.add_column("#", style="dim")
    table.add_column("Model", style="yellow")

    for metric in report.metrics:
        table.add_column(metric, justify="left", width=max_widths[metric])

    table.add_column("Average", justify="right", style="bold cyan")

    model_averages = {}
    for model_name in report.model_names:
        avg_score = np.mean(
            [report.results[model_name][metric] for metric in report.metrics]
        )
        model_averages[model_name] = avg_score

    sorted_models = sorted(model_averages.items(), key=lambda x: x[1], reverse=True)
    best_model_name = sorted_models[0][0]

    for model_name, avg_score in sorted_models:
        model_name_cell_value = model_name
        if model_name == best_model_name:
            model_name_cell_value = (
                f"[bold bright_green]{model_name}[/bold bright_green]"
            )

        row_data: list[Any] = [
            f"[dim]{model_to_letter[model_name]}[/dim]",
            model_name_cell_value,
        ]

        for metric in report.metrics:
            cell_text = Text()
            score = report.results[model_name][metric]
            score_part, superscript_part = cell_data[model_name][metric]

            if score == max_scores[metric]:
                cell_text.append(score_part, style="bold bright_green")
            elif score == min_scores[metric]:
                cell_text.append(score_part, style="bold bright_red")
            else:
                cell_text.append(score_part)

            if superscript_part:
                cell_text.append(" ")
                cell_text.append(superscript_part, style="bold bright_cyan")

            row_data.append(cell_text)

        row_data.append(f"{avg_score:.4f}")
        table.add_row(*row_data)

    console.print(table)

### 1. Questions (English) & Chunks (English)

In [22]:
runs_english = []
models_to_benchmark = list(chunks_english[0]["embeddings"].keys())
for current_model_name in models_to_benchmark:
    run = create_run_from_data(
        chunks=chunks_english,
        questions=questions_english,
        model_name=current_model_name,
    )
    runs_english.append(run)
qrels_english = create_qrels_from_data(questions_english)

metrics = ["mrr", "recall@1", "recall@5", "ndcg@5"]
report_english_english = compare(
    qrels=qrels_english,
    runs=runs_english,
    metrics=metrics,
    max_p=0.05,
    stat_test="fisher",
)
print_benchmark_report(report_english_english)

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┓
┃ # ┃ Model                  ┃ mrr         ┃ recall@1     ┃ recall@5    ┃ ndcg@5     ┃ Average ┃
┣━━━╋━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━┫
┃ g ┃ qwen3-embedding-4b     ┃ 0.7425 abce ┃ 0.6361 abcef ┃ 0.8784 abc  ┃ 0.7675 abc ┃  0.7561 ┃
┃ d ┃ gemini-embedding-001   ┃ 0.7375 abc  ┃ 0.6303 abc   ┃ 0.8792 abc  ┃ 0.7639 abc ┃  0.7527 ┃
┃ f ┃ qwen3-embedding-8b     ┃ 0.7328 abc  ┃ 0.6129 abc   ┃ 0.8941 abcg ┃ 0.7643 abc ┃  0.7510 ┃
┃ e ┃ qwen3-embedding-0.6b   ┃ 0.7276 abc  ┃ 0.6112 abc   ┃ 0.8801 abc  ┃ 0.7560 abc ┃  0.7437 ┃
┃ b ┃ text-embedding-3-large ┃ 0.6994 c    ┃ 0.5773 c     ┃ 0.8536 c    ┃ 0.7271 c   ┃  0.7144 ┃
┃ a ┃ text-embedding-3-small ┃ 0.6972 c    ┃ 0.5790 c     ┃ 0.8544 c    ┃ 0.7261 c   ┃  0.7142 ┃
┃ c ┃ all-minilm-l6-v2       ┃ 0.5396      ┃ 0.4268       ┃ 0.6576      ┃ 0.5501     ┃  0.5435 ┃
┗━━━┻━━━━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━┛

### 2. Questions (Arabic) & Chunks (English)

In [23]:
runs_english = []
models_to_benchmark = list(chunks_english[0]["embeddings"].keys())
for current_model_name in models_to_benchmark:
    run = create_run_from_data(
        chunks=chunks_english,
        questions=questions_arabic,
        model_name=current_model_name,
    )
    runs_english.append(run)
qrels_arabic = create_qrels_from_data(questions_arabic)

metrics = ["mrr", "recall@1", "recall@5", "ndcg@5"]
report_arabic_english = compare(
    qrels=qrels_arabic,
    runs=runs_english,
    metrics=metrics,
    max_p=0.05,
    stat_test="fisher",
)
print_benchmark_report(report_arabic_english)

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ # ┃ Model                  ┃ mrr          ┃ recall@1     ┃ recall@5     ┃ ndcg@5        ┃ Average ┃
┣━━━╋━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━╋━━━━━━━━━┫
┃ d ┃ gemini-embedding-001   ┃ 0.7075 abcef ┃ 0.5881 abcef ┃ 0.8594 abceg ┃ 0.7350 abcefg ┃  0.7225 ┃
┃ g ┃ qwen3-embedding-4b     ┃ 0.6907 abce  ┃ 0.5757 abce  ┃ 0.8313 abce  ┃ 0.7142 abce   ┃  0.7030 ┃
┃ f ┃ qwen3-embedding-8b     ┃ 0.6811 abce  ┃ 0.5583 abce  ┃ 0.8395 abce  ┃ 0.7084 abce   ┃  0.6968 ┃
┃ b ┃ text-embedding-3-large ┃ 0.6297 ac    ┃ 0.4938 ac    ┃ 0.8007 ac    ┃ 0.6584 ac     ┃  0.6456 ┃
┃ e ┃ qwen3-embedding-0.6b   ┃ 0.6145 ac    ┃ 0.4864 ac    ┃ 0.7816 ac    ┃ 0.6411 ac     ┃  0.6309 ┃
┃ a ┃ text-embedding-3-small ┃ 0.5193 c     ┃ 0.3838 c     ┃ 0.6675 c     ┃ 0.5346 c      ┃  0.5263 ┃
┃ c ┃ all-minilm-l6-v2       ┃ 0.1086       ┃ 0.0380       ┃ 0.1563       ┃ 0.0966        ┃  0.0999 ┃
┗━━━┻━━━━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━━┻━━━━━━━━━┛

### 3. Questions (English) & Chunks (Arabic)

In [24]:
runs_arabic = []
models_to_benchmark = list(chunks_english[0]["embeddings"].keys())
for current_model_name in models_to_benchmark:
    run = create_run_from_data(
        chunks=chunks_arabic,
        questions=questions_english,
        model_name=current_model_name,
    )
    runs_arabic.append(run)
qrels_english = create_qrels_from_data(questions_english)

metrics = ["mrr", "recall@1", "recall@5", "ndcg@5"]
report_english_arabic = compare(
    qrels=qrels_english,
    runs=runs_arabic,
    metrics=metrics,
    max_p=0.05,
    stat_test="fisher",
)
print_benchmark_report(report_english_arabic)

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ # ┃ Model                  ┃ mrr          ┃ recall@1     ┃ recall@5     ┃ ndcg@5       ┃ Average ┃
┣━━━╋━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━┫
┃ f ┃ qwen3-embedding-8b     ┃ 0.7288 abceg ┃ 0.6121 abceg ┃ 0.8867 abceg ┃ 0.7593 abceg ┃  0.7467 ┃
┃ d ┃ gemini-embedding-001   ┃ 0.7280 abceg ┃ 0.6179 abceg ┃ 0.8701 abce  ┃ 0.7537 abceg ┃  0.7424 ┃
┃ g ┃ qwen3-embedding-4b     ┃ 0.7042 abce  ┃ 0.5864 abce  ┃ 0.8553 abce  ┃ 0.7309 abce  ┃  0.7192 ┃
┃ b ┃ text-embedding-3-large ┃ 0.6248 ac    ┃ 0.5004 ac    ┃ 0.7709 ac    ┃ 0.6451 ac    ┃  0.6353 ┃
┃ e ┃ qwen3-embedding-0.6b   ┃ 0.6158 ac    ┃ 0.4847 ac    ┃ 0.7758 ac    ┃ 0.6401 ac    ┃  0.6291 ┃
┃ a ┃ text-embedding-3-small ┃ 0.5232 c     ┃ 0.3805 c     ┃ 0.6964 c     ┃ 0.5469 c     ┃  0.5368 ┃
┃ c ┃ all-minilm-l6-v2       ┃ 0.0952       ┃ 0.0306       ┃ 0.1332       ┃ 0.0800       ┃  0.0848 ┃
┗━━━┻━━━━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━┛

### 4. Questions (Arabic) & Chunks (Arabic)

In [25]:
runs_arabic = []
models_to_benchmark = list(chunks_english[0]["embeddings"].keys())
for current_model_name in models_to_benchmark:
    run = create_run_from_data(
        chunks=chunks_arabic,
        questions=questions_arabic,
        model_name=current_model_name,
    )
    runs_arabic.append(run)
qrels_arabic = create_qrels_from_data(questions_arabic)

metrics = ["mrr", "recall@1", "recall@5", "ndcg@5"]
report_arabic_arabic = compare(
    qrels=qrels_arabic,
    runs=runs_arabic,
    metrics=metrics,
    max_p=0.05,
    stat_test="fisher",
)
print_benchmark_report(report_arabic_arabic)

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ # ┃ Model                  ┃ mrr          ┃ recall@1     ┃ recall@5     ┃ ndcg@5       ┃ Average ┃
┣━━━╋━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━┫
┃ d ┃ gemini-embedding-001   ┃ 0.7236 abceg ┃ 0.6121 abceg ┃ 0.8685 abceg ┃ 0.7498 abceg ┃  0.7385 ┃
┃ f ┃ qwen3-embedding-8b     ┃ 0.7167 abceg ┃ 0.5997 abceg ┃ 0.8768 abceg ┃ 0.7469 abceg ┃  0.7350 ┃
┃ g ┃ qwen3-embedding-4b     ┃ 0.6964 abce  ┃ 0.5782 abce  ┃ 0.8470 abce  ┃ 0.7228 abce  ┃  0.7111 ┃
┃ e ┃ qwen3-embedding-0.6b   ┃ 0.6274 abc   ┃ 0.5012 abc   ┃ 0.7858 abc   ┃ 0.6523 abc   ┃  0.6417 ┃
┃ b ┃ text-embedding-3-large ┃ 0.5824 ac    ┃ 0.4491 ac    ┃ 0.7461 ac    ┃ 0.6059 ac    ┃  0.5959 ┃
┃ a ┃ text-embedding-3-small ┃ 0.5401 c     ┃ 0.4036 c     ┃ 0.6940 c     ┃ 0.5605 c     ┃  0.5495 ┃
┃ c ┃ all-minilm-l6-v2       ┃ 0.0751       ┃ 0.0207       ┃ 0.0993       ┃ 0.0602       ┃  0.0638 ┃
┗━━━┻━━━━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━┻━━━━━━━━━┛